In [19]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as plt

#Import cranial database 
df = pd.read_csv('../../datasources/fdb/postcranial.csv')
#This shuffles the rows.
df = df.sample(frac=1)

#Drop categorical data. We'll focus only on measurement data.
del df['Item']
del df['ID']
del df['ContNum']
del df['FDN']
del df['Pop']
del df['PopSex']
del df['BirthYear']
del df['Age'] #Probably really important.
del df['Comments']
del df['DB']

#I don't know what these are.
# del df['FStat']
# del df['CStat']
# del df['MStat']

rows, cols = df.shape 
print("Starting with", rows, "individuals and", cols, "features.")

Starting with 3861 individuals and 48 features.


In [20]:
# df = df.dropna(subset = ['Pop'])
# df['Pop'].value_counts(dropna=False)

In [21]:
#Turn pop to one hot.
# df['Pop'] = df['Pop'].map({'W':0, 'B':1, 'H':2})
# cat_col = pd.DataFrame(df['Pop'])
# from sklearn import preprocessing
# enc = preprocessing.OneHotEncoder()
# enc.fit(cat_col)
# cat_col_oh = pd.DataFrame(enc.transform(cat_col).toarray())
# del df['Pop']
# df = pd.concat([df, cat_col_oh], axis=1)

In [22]:
#Consider our target col, Sex.
print("Unique entries.")
df['Sex'].value_counts(dropna=False)

Unique entries.


M      2973
F       887
NaN       1
Name: Sex, dtype: int64

In [23]:
#Clean N and Nan entries
df = df.dropna(subset = ['Sex'])

#Move from object to float.
df['Sex'] = df['Sex'].map({'M':1.0, 'F':0.})
df.describe()

/usr/local/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Sex,FStat,CStat,MStat,claxln,claapd,clavrd,scapht,scapbr,humxln,...,tibxln,tibpeb,tibdeb,tibnfx,tibnft,tibcir,fibxln,fibmdm,calcxl,calcbr
count,3860.000000,894.000000,1033.000000,1211.000000,1635.000000,1570.000000,1568.000000,1639.000000,1705.000000,3572.000000,...,3495.000000,1602.000000,1648.000000,1739.000000,1721.000000,1498.000000,3304.000000,1522.000000,1447.000000,1391.000000
mean,0.770207,171.201342,168.998064,174.027250,150.682569,12.415924,10.190689,154.868212,103.936070,328.865062,...,385.788555,76.134207,50.015777,34.504313,24.496223,93.779039,377.056598,15.296978,82.938493,42.479511
std,0.420754,9.649151,9.381864,6.279055,11.852631,2.028384,1.794479,13.947257,8.539951,20.914442,...,28.831919,6.111199,4.387955,3.831515,3.029720,9.349371,26.095440,2.008818,6.177163,3.729444
min,0.000000,137.000000,134.000000,156.000000,118.000000,7.000000,4.000000,98.000000,83.000000,236.000000,...,268.000000,27.000000,38.000000,22.000000,17.000000,68.000000,276.000000,9.000000,65.000000,28.000000
25%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.000000,202.000000,203.000000,191.000000,183.000000,22.000000,20.000000,199.000000,131.000000,404.000000,...,483.000000,94.000000,67.000000,55.000000,41.000000,127.000000,469.000000,22.000000,100.000000,83.000000


In [24]:
#Split into data and labels.
X = df.drop(['Sex'], axis=1)
y = df[['Sex']]

In [25]:
#Split train test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

In [26]:
X_train.isnull().sum().sum(),X_train.isnull().sum().sum()

(94705, 94705)

In [27]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3821, 47), (39, 47), (3821, 1), (39, 1))

In [28]:
#Standardize df. Can't use scikit's scale because it doesn't like Nan.
X_train_mean = X_train.mean()
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / (X_train_std)
#Intentionally using train stats for test set.
X_test = ( X_test - X_train_mean) / (X_train_std)
X_train.describe()

/usr/local/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,FStat,CStat,MStat,claxln,claapd,clavrd,scapht,scapbr,humxln,humebr,...,tibxln,tibpeb,tibdeb,tibnfx,tibnft,tibcir,fibxln,fibmdm,calcxl,calcbr
count,8.880000e+02,1.024000e+03,1.198000e+03,1.623000e+03,1.559000e+03,1.557000e+03,1.626000e+03,1.692000e+03,3.534000e+03,1.749000e+03,...,3.459000e+03,1.589000e+03,1.634000e+03,1.725000e+03,1.707000e+03,1.485000e+03,3.274000e+03,1.510000e+03,1.435000e+03,1.379000e+03
mean,-8.301668e-16,-3.469447e-18,1.144697e-15,6.435600e-16,-9.457192e-17,-2.424379e-16,-5.702695e-16,1.469799e-17,-8.117760e-16,3.818812e-16,...,6.275536e-16,3.141323e-16,5.892200e-16,8.361749e-16,-2.705640e-16,-5.909227e-16,6.554182e-16,3.882104e-16,5.818033e-16,3.065794e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.541716e+00,-3.723138e+00,-2.870149e+00,-2.758188e+00,-2.667449e+00,-3.446234e+00,-4.086327e+00,-2.456239e+00,-4.430813e+00,-2.911210e+00,...,-4.086069e+00,-8.039875e+00,-2.740558e+00,-3.258708e+00,-2.477778e+00,-2.757500e+00,-3.872075e+00,-3.138463e+00,-2.914421e+00,-3.882950e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,3.192188e+00,3.628249e+00,2.702427e+00,2.730250e+00,4.726849e+00,5.462408e+00,2.887764e+00,3.175933e+00,3.592448e+00,2.376891e+00,...,3.377923e+00,2.922028e+00,3.871667e+00,5.340493e+00,5.449563e+00,3.549399e+00,3.527464e+00,3.335689e+00,2.770998e+00,1.085986e+01


In [29]:
#The imputer. I haven't looked closely at this algo. Try other variations.
from fancyimpute import KNN, SoftImpute, NuclearNormMinimization, MatrixFactorization, BiScaler
if True:
#     X_train_filled = pd.DataFrame(KNN(k=1).complete(X_train))
#     X_test_filled = pd.DataFrame(KNN(k=1).complete(X_test))
    X_train_filled = SoftImpute().complete(X_train)
    X_test_filled = SoftImpute().complete(X_test)
    
else:
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    X_train_filled = pd.DataFrame(imp.fit_transform(X_train_filled))
    # X_train_filled = pd.DataFrame(imp.transform(X_train_filled))
    X_test_filled = pd.DataFrame(imp.transform(X_test_filled))

[SoftImpute] Max Singular Value of X_init = 199.849025
[SoftImpute] Iter 1: observed MAE=0.066610 rank=47
[SoftImpute] Iter 2: observed MAE=0.067231 rank=47
[SoftImpute] Iter 3: observed MAE=0.067797 rank=47
[SoftImpute] Iter 4: observed MAE=0.068284 rank=47
[SoftImpute] Iter 5: observed MAE=0.068641 rank=46
[SoftImpute] Iter 6: observed MAE=0.068860 rank=46
[SoftImpute] Iter 7: observed MAE=0.069047 rank=46
[SoftImpute] Iter 8: observed MAE=0.069201 rank=46
[SoftImpute] Iter 9: observed MAE=0.069326 rank=46
[SoftImpute] Iter 10: observed MAE=0.069427 rank=46
[SoftImpute] Iter 11: observed MAE=0.069510 rank=46
[SoftImpute] Iter 12: observed MAE=0.069577 rank=46
[SoftImpute] Iter 13: observed MAE=0.069633 rank=46
[SoftImpute] Iter 14: observed MAE=0.069679 rank=46
[SoftImpute] Iter 15: observed MAE=0.069717 rank=46
[SoftImpute] Iter 16: observed MAE=0.069749 rank=46
[SoftImpute] Iter 17: observed MAE=0.069776 rank=46
[SoftImpute] Iter 18: observed MAE=0.069799 rank=46
[SoftImpute] Iter 

In [30]:
#Num zero entries
(X_train_filled == 0).astype(int).sum().sum(),(X_test_filled == 0).astype(int).sum().sum()

(1739, 0)

In [31]:
#Impute zeros with mean. Could do this better.
#Think this introduces an improbable bug when the train set
#Has representation in a col that the test set doesn't or 
#Vice versa. Ignoring it for now. 
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=0, strategy='mean', axis=0)
X_train_filled = pd.DataFrame(imp.fit_transform(X_train_filled))
# X_train_filled = pd.DataFrame(imp.transform(X_train_filled))
X_test_filled = pd.DataFrame(imp.transform(X_test_filled))

In [32]:
#No zero values
(X_train_filled == 0).astype(int).sum().sum(), (X_test_filled == 0).astype(int).sum().sum()

(0, 0)

In [33]:
#Notice that the training (and testing) set is no longer standardized
X_train_filled.describe()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
count,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,...,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000
mean,0.021980,0.048837,-0.106184,0.036019,0.014172,0.006317,0.029426,0.034375,-0.007121,0.017290,...,-0.010971,0.012566,0.008560,0.020632,0.020641,0.018765,-0.007288,0.001610,0.021614,0.011469
std,0.692291,0.751864,0.671431,0.705573,0.670210,0.655923,0.698010,0.714040,0.976839,0.702737,...,0.972755,0.693421,0.680848,0.698156,0.691293,0.692265,0.968754,0.645103,0.688786,0.660751
min,-3.541716,-3.723138,-2.870149,-2.758188,-2.667449,-3.446234,-4.086327,-2.456239,-4.430813,-2.911210,...,-4.086069,-8.039875,-2.740558,-3.258708,-2.477778,-2.757500,-4.256840,-3.138463,-2.914421,-3.882950
25%,-0.331405,-0.364986,-0.481902,-0.253130,-0.202683,-0.105493,-0.207330,-0.232422,-0.657970,-0.210355,...,-0.649161,-0.186572,-0.195654,-0.192074,-0.166229,-0.228871,-0.613211,-0.150393,-0.293566,-0.228130
50%,0.077033,0.112368,-0.139540,0.064444,0.032067,0.006379,0.069513,0.074009,0.010635,0.054517,...,-0.010971,0.050186,0.021387,0.047292,0.042137,0.043087,0.000222,0.008416,0.054552,0.047837
75%,0.418807,0.511793,0.211390,0.366000,0.283152,0.118563,0.358143,0.359847,0.631483,0.320407,...,0.635340,0.304260,0.223542,0.257470,0.215417,0.284152,0.613656,0.121302,0.334390,0.264811
max,3.192188,3.628249,2.702427,2.730250,4.726849,5.462408,2.887764,3.175933,3.592448,2.376891,...,3.377923,2.922028,3.871667,5.340493,5.449563,3.549399,3.527464,3.335689,2.770998,10.859860


In [34]:
#Should we standardize again?
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train_filled)
X_train_filled = pd.DataFrame(scaler.transform(X_train_filled))
#Uses test stats
X_test_filled = pd.DataFrame(scaler.transform(X_test_filled))

In [35]:
#Save to disk
X_train_filled.to_csv("data/X_train_preproc.csv", index=False)
#Ravel strips col name and reindexes
pd.DataFrame(np.ravel(y_train)).to_csv("data/y_train_preproc.csv", index=False)

X_test_filled.to_csv("data/X_test_preproc.csv", index=False)
pd.DataFrame(np.ravel(y_test)).to_csv("data/y_test_preproc.csv", index=False)

In [36]:
X_test_filled

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,0.227054,0.846534,2.286279,0.536281,0.247454,0.188446,0.431393,0.429588,1.729548,-0.114072,...,0.735886,0.492741,0.254841,0.140370,0.429859,0.252748,0.363986,0.349982,0.303710,-0.112374
1,0.154312,0.228828,1.337635,0.108591,0.142648,0.224607,0.497757,0.417756,0.702727,0.240313,...,1.235592,0.250512,0.182767,0.244727,0.104028,0.221128,0.997287,0.274489,0.354501,0.077122
2,0.113557,0.131535,-0.796813,0.276151,0.209556,0.240375,0.188766,0.234778,0.360454,0.220332,...,0.236181,0.167807,0.311023,0.141768,0.114614,0.142666,0.095848,0.228835,0.305484,0.102700
3,-0.151246,-0.374567,-1.745457,-0.063204,-0.011033,-0.018377,-0.318361,-0.246101,-0.470782,-0.005393,...,-0.584763,-0.184597,0.094568,-0.099370,-0.125254,-0.110683,-0.704711,-0.070294,-0.121681,0.014342
4,-1.455261,-1.464307,-0.418760,-2.644145,-1.059221,-1.868437,-0.540271,-2.009370,-1.839877,-1.659158,...,-2.012493,-2.646995,-1.693792,-1.711424,-2.658855,-2.621009,-2.010896,-0.623188,-3.083848,0.191242
5,0.774736,1.076564,0.671405,0.557417,0.826043,0.461640,1.281321,0.563001,1.142794,0.859738,...,0.557420,0.807072,1.000250,0.575731,0.413999,0.531822,1.155613,0.627367,0.857819,0.211663
6,2.484582,0.782264,0.439467,1.664617,1.883244,-0.170484,4.507590,2.428075,1.631756,1.685726,...,1.449751,1.364624,2.995252,1.648202,2.119860,0.931016,1.353520,1.308521,1.397625,-1.025946
7,0.035881,0.093374,-0.559652,0.031644,0.213033,0.101635,0.189346,0.057765,0.213765,0.243512,...,-0.085058,0.057378,0.323744,0.141988,-0.043030,0.074470,0.060746,0.126697,0.136600,0.068749
8,0.593843,0.436269,-0.559652,0.849264,0.462961,0.569310,0.685917,0.750876,0.849416,0.500791,...,1.342672,0.727907,0.553530,0.320834,0.598126,0.456976,0.734246,0.656353,0.892072,0.222144
9,0.216615,-1.049267,-0.122441,-0.432552,-0.111816,-0.413437,0.339745,-0.200074,-0.666367,0.310791,...,0.164795,0.143794,0.473259,0.018591,-0.245523,-0.065980,0.245242,0.000041,-0.006449,0.057917
